<a href="https://colab.research.google.com/github/analyticswithadam/Python/blob/main/GPT_4o_Supplier_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# get the openai secret key
import pandas
import getpass
secret_key = getpass.getpass('Please enter your openai key: ')
from IPython.display import display, Markdown

Please enter your openai key: ··········


In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00


In [3]:
%pip install alpha_vantage

In [4]:
import requests
import json
from IPython.display import display, Markdown

api_key = 'GET API KEY'
ticker = 'DIS'  # Example ticker

# Function to fetch financial data
def fetch_financial_data(function, ticker, api_key):
    url = f"https://www.alphavantage.co/query?function={function}&symbol={ticker}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching {function} data.")
        return None

# Fetch income statement
income_statement = fetch_financial_data("INCOME_STATEMENT", ticker, api_key)



# Fetch balance sheet
balance_sheet = fetch_financial_data("BALANCE_SHEET", ticker, api_key)

In [5]:
# Assuming you've already fetched the balance sheet data as 'balance_sheet'
quarterly_income_reports = income_statement['quarterlyReports'][:4]
quarterly_balance_reports = balance_sheet['quarterlyReports'][:4]

formatted_data = {"financialData": []}

# Iterate through the income reports and balance reports simultaneously
for income_report, balance_report in zip(quarterly_income_reports, quarterly_balance_reports):
    # Filter out entries with values "None" or an empty string for income statement
    formatted_income_report = {k: v for k, v in income_report.items() if v not in ["None", ""]}
    # Filter out entries with values "None" or an empty string for balance sheet
    formatted_balance_report = {k: v for k, v in balance_report.items() if v not in ["None", ""]}
    # Combine the data from both reports
    formatted_report = {**formatted_income_report, **formatted_balance_report}
    # Append the combined report to the financialData list
    formatted_data["financialData"].append(formatted_report)

# Convert the combined data to a JSON string
out_string = json.dumps(formatted_data, indent=4)
print(out_string)

{
    "financialData": [
        {
            "fiscalDateEnding": "2024-03-31",
            "reportedCurrency": "USD",
            "grossProfit": "15910000000",
            "totalRevenue": "22083000000",
            "costOfRevenue": "6173000000",
            "costofGoodsAndServicesSold": "15398000000",
            "operatingIncome": "3845000000",
            "sellingGeneralAndAdministrative": "3790000000",
            "operatingExpenses": "6667000000",
            "investmentIncomeNet": "87000000",
            "netInterestIncome": "-311000000",
            "interestIncome": "190000000",
            "interestExpense": "501000000",
            "otherNonOperatingIncome": "0",
            "depreciationAndAmortization": "434000000",
            "incomeBeforeTax": "421000000",
            "incomeTaxExpense": "441000000",
            "interestAndDebtExpense": "501000000",
            "netIncomeFromContinuingOperations": "216000000",
            "comprehensiveIncomeNetOfTax": "-27000000",
   

In [9]:
from openai import OpenAI
client = OpenAI(api_key=secret_key)

def get_response(prompt):
  # Create a request to the chat completions endpoint
  response = client.chat.completions.create(
    model="gpt-4o",
    # Assign the role and content for the message
    messages=
     [
        {
            "role": "system", "content": """

              Role
              -------
              You are a CPA Employed in the Finance Function, tasked with assessing supplier risk.

              Task
              ------
              Analyze the provided 4 quarters of financial data.
              Compare the financials between the four quarters, noting any significant movements or trends.
              Evaluate the implications of these movements for the company's financial health and operational efficiency.

              From the perspective of a company reciving services from this company highlight key risks in increasing purchases
              from this purchaser based on what you have gathered from the financials.

              Decide whether the team need to investigate this supplier further for risk backed up the financials.

              Output
              ------
              Ensure the analysis reflects the specific relevant movements across the quarters,
              highlighting significant changes relevant to their viability as a major supplier.
              The output should strictly adhere to the data provided, avoiding assumptions or inferences not directly supported by the numbers.
              Im printing in markdown in python ensure summary tables readible and numeric text is legible
              Do not include any currency symbols, use 22.5M USD instead of $22.5M, display figures in millions and use thousand seperators where appropriate ie 23,100M


              Data
              -----

            """
        },
        {
            "role": "user", "content": prompt
        }
     ],
    temperature = 0.4,
    max_tokens=3000)
  return response.choices[0].message.content

In [10]:
output = get_response(out_string)

In [11]:
display(Markdown(output))

## Financial Analysis of Supplier

### Summary of Financial Data

| Fiscal Date Ending | Total Revenue (M) | Gross Profit (M) | Operating Income (M) | Net Income (M) | Total Assets (M) | Total Liabilities (M) | Total Shareholder Equity (M) |
|--------------------|-------------------|------------------|----------------------|----------------|------------------|-----------------------|------------------------------|
| 2024-03-31         | 22,083            | 15,910           | 3,845                | -20            | 195,110          | 95,858                | 99,252                       |
| 2023-12-31         | 23,549            | 19,415           | 3,876                | 1,911          | 197,774          | 97,053                | 100,721                      |
| 2023-09-30         | 21,241            | 16,069           | 2,976                | 264            | 205,579          | 106,302               | 99,277                       |
| 2023-06-30         | 22,330            | 15,461           | 3,559                | -460           | 203,783          | 106,173               | 97,610                       |

### Key Financial Movements and Trends

1. **Revenue and Profitability:**
   - **Revenue:** The revenue fluctuated across the quarters, with a peak in Q4 2023 (23,549M) and a dip in Q3 2023 (21,241M). The latest quarter (Q1 2024) shows a slight recovery to 22,083M.
   - **Gross Profit:** Gross profit followed a similar trend, peaking in Q4 2023 (19,415M) and dipping in Q3 2023 (16,069M). The latest quarter (Q1 2024) shows a gross profit of 15,910M.
   - **Operating Income:** Operating income remained relatively stable, with minor fluctuations. The highest was in Q4 2023 (3,876M), and the lowest was in Q3 2023 (2,976M).
   - **Net Income:** Net income showed significant volatility, with a notable loss in Q2 2023 (-460M) and a peak in Q4 2023 (1,911M). The latest quarter (Q1 2024) reported a slight loss (-20M).

2. **Assets and Liabilities:**
   - **Total Assets:** Total assets showed a slight decrease from Q3 2023 (205,579M) to Q1 2024 (195,110M).
   - **Total Liabilities:** Total liabilities decreased from Q3 2023 (106,302M) to Q1 2024 (95,858M), indicating some deleveraging.
   - **Total Shareholder Equity:** Shareholder equity remained relatively stable, with minor fluctuations, ending at 99,252M in Q1 2024.

3. **Cash and Receivables:**
   - **Cash and Cash Equivalents:** There was a significant decrease in cash from Q3 2023 (14,182M) to Q1 2024 (6,635M).
   - **Current Net Receivables:** Receivables decreased from Q4 2023 (14,115M) to Q1 2024 (12,026M).

4. **Debt:**
   - **Current Debt:** Current debt increased from Q2 2023 (2,645M) to Q1 2024 (6,789M).
   - **Long-term Debt:** Long-term debt remained significant, with a slight decrease from Q3 2023 (46,431M) to Q1 2024 (39,510M).

### Implications for Financial Health and Operational Efficiency

- **Revenue and Profitability Volatility:** The fluctuations in revenue and profitability indicate potential instability in the company's operations or market conditions. The recent losses in Q2 2023 and Q1 2024 are concerning.
- **Decreasing Cash Reserves:** The significant decrease in cash reserves could impact the company's liquidity and its ability to meet short-term obligations.
- **High Debt Levels:** The high levels of both current and long-term debt pose a risk, especially with the increasing current debt. This could strain the company's cash flow and financial stability.
- **Decreasing Receivables:** The decrease in receivables suggests potential issues in collecting payments from customers, which could further strain liquidity.

### Key Risks in Increasing Purchases from this Supplier

1. **Financial Instability:** The volatility in revenue and profitability, along with recent losses, indicates potential financial instability.
2. **Liquidity Concerns:** The significant decrease in cash reserves and high current debt levels raise concerns about the company's ability to meet short-term obligations.
3. **High Debt Levels:** The substantial long-term debt could impact the company's financial flexibility and increase the risk of default.
4. **Decreasing Receivables:** Potential issues in collecting payments could affect the company's cash flow and operational efficiency.

### Recommendation

Given the observed financial instability, liquidity concerns, high debt levels, and potential issues with receivables, it is advisable to investigate this supplier further for risk. The financial data suggests that there are significant risks in increasing purchases from this supplier without a thorough risk assessment.